In [1]:
import geopandas as gpd
import numpy as np

## Pre-processing 

### Perimeter Data Exploration

In [4]:
shapefile = gpd.read_file("ca_camp_20181108_1754_dd83.shp")
shapefile.shape
shapefile["perDatTime"][0]
#print(shapefile)
for i in range(24):
    print(shapefile.columns[i])

shapefile2 = gpd.read_file("./2009_Fires/2009_Fires.shp")
shapefile3 = gpd.read_file("./US_HIST_FIRE_PERIMTRS_DD83-3/US_HIST_FIRE_PERIMTRS_DD83.shp")
#print(shapefile2)

##Visualizing columns
shapefile3 = gpd.read_file("./2000_perimeters_dd83/2000_perimeters_dd83.shp")
print(shapefile3.columns)
print(shapefile3)

### Web Scraping with BeautifulSoup

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [6]:
url = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/2019_fire_data'
url2 = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/2019_fire_data/Alabama/'
https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/2019_fire_data/Alabama/
response = requests.get(url2)

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
soup.findAll('a')
one_a_tag = soup.findAll('a')[0]
link = one_a_tag['href']

In [11]:
#get list of urls of fire perimeters for all states for 2019
def getStateUrls():
    url = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/2019_fire_data'
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    #soup.findAll('a')

    state_urls = []
    line_count = 0
    for i,eachtag in enumerate(soup.findAll('a')):
        if i >0:
            statelink = eachtag['href']
            state_url = 'https://rmgsc.cr.usgs.gov' + statelink
            state_urls.append(state_url)
    return state_urls
        


In [50]:
state_urls = getStateUrls()

#clean some stateurls
prob1 = state_urls[6]
prob2 = state_urls[13]
prob3 = state_urls[14]
prob4 = state_urls[9]
state_urls.remove(prob1)
state_urls.remove(prob2)
state_urls.remove(prob3)
state_urls.remove(prob4)

#### Downloading all perimeter files for 2019 across all state urls

In [ ]:
#This assumes you create a 'GeomacData' folder in your current directory to store these files.
for eachstateurl in state_urls:
    urlnew = eachstateurl
    response = requests.get(urlnew)
    soup = BeautifulSoup(response.text, "html.parser")
    suburls_hrefs = soup.findAll('a')
    suburls_actual = [element['href'] for element in suburls_hrefs] 
    for i,suburl in enumerate(suburls_actual):    #for each fire url in state folder
        if i>0:
            #print(suburl)
            firelink = 'https://rmgsc.cr.usgs.gov' + suburl
            response = requests.get(firelink)
            soup = BeautifulSoup(response.text, "html.parser")  
            fire_evols = soup.findAll('a')   #list of shape files links in the fire's page.
            fire_shp_files = [element['href'] for element in fire_evols] 
            for j,fire_evol in enumerate(fire_shp_files):
                if j>0:
                    download_url = 'https://rmgsc.cr.usgs.gov' + fire_evol
                    storeurl = './GeomacData/'+ fire_evol.split("/")[-1]
                    urllib.request.urlretrieve(download_url,storeurl) 
                    time.sleep(1)
                

In [ ]:
### Reading the data

In [ ]:
from pathlib import Path
import pandas
import geopandas

folder = Path("./GeomacData")
shapefiles = folder.glob("*.shp")

In [ ]:
for shp in shapefiles:
        print(shp)

### Merging the scraped data

In [ ]:
import os
import geopandas as gpd
import pandas as pd

file = os.listdir("./GeomacData")
path = [os.path.join("./GeomacData", i) for i in file if i.endswith('.shp')]
#tester =[each for each in os.listdir(folder) if each.endswith('.shp')]
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)




In [ ]:
gdf2 = gdf.copy()  #duplicate

### CLEANING COLUMNS OF MERGED DATA.
#df['feedback_id'].combine_first(df['_id'])
gdf2['perDatTime'] = gdf2['perDatTime'].combine_first(gdf['PERDATTIME'])
gdf2['fireName'] = gdf2['perDatTime'].combine_first(gdf['PERDATTIME'])
gdf2['fireNum'] = gdf2['perDatTime'].combine_first(gdf['PERDATTIME'])
gdf2['fireYear'] = gdf2['perDatTime'].combine_first(gdf['PERDATTIME'])
gdf2['irwinid'] = gdf2['irwinid'].combine_first(gdf['IRWINID'])
gdf2['state'] = gdf2['state'].combine_first(gdf['STATE'])
gdf2['gisAcres'] = gdf2['gisAcres'].combine_first(gdf['GISACRES'])
gdf2['incidentID'] = gdf2['incidentID'].combine_first(gdf['INCIDENTID'])

In [ ]:
gdf2.to_file("results.shp")

## Working with saved Perimeter File

In [2]:
gp = gpd.read_file("results.shp")

In [3]:
gp.columns   #see column names, sometimes they change

Index(['ACTIVE', 'AGENCY', 'COMMENTS', 'COMPFIRECD', 'COMPLEXNM', 'COMPPARID',
       'ComplexN_1', 'DATECRNT', 'FIRECODE', 'FIRENAME', 'FIRENUM', 'FIREYEAR',
       'GISACRES', 'INCIDENTID', 'INCIWEBID', 'INCOMPLEX', 'IRWINID', 'LATEST',
       'MAPMETHOD', 'MERGEID', 'OBJECTID', 'PERDATTIME', 'SHAPE_AREA',
       'SHAPE_Ar_1', 'STATE', 'Shape_Ar_2', 'UNITIDOWN', 'UNITIDPROT',
       'active_1', 'agency_1', 'comments_1', 'compParI_1', 'compfire_1',
       'dateCrnt_1', 'fireName_1', 'fireNum_1', 'fireYear_1', 'firecode_1',
       'gisAcres_1', 'inComple_1', 'incident_1', 'inciwebI_1', 'irwinid_1',
       'latest_1', 'mapmetho_1', 'mergeid_1', 'objectid_1', 'perDatTi_1',
       'state_1', 'unitIDOw_1', 'unitIDPr_1', 'geometry'],
      dtype='object')

In [ ]:
#gp['geometry'].bounds

In [4]:
gp.head()['perDatTi_1'][0]

'8/21/2019 4:29:27 PM'

In [5]:
a = gp.head()['perDatTi_1'][0]
year = a[:9]
time = a[10:]

## Downloading Sentinel Images

In [6]:
from owslib.wms import WebMapService

In [7]:
wms = WebMapService('http://services.sentinel-hub.com/ogc/wms/45271f49-0dcb-4329-8b35-8b6508fee50b')

### Preliminary exploration of WMS options/features

In [8]:

wms.identification.type
wms.identification.version
wms.identification.title
wms.identification.abstract

'The Copernicus project’s Sentinel satellites are revolutionizing earth observation (EO). Its free, full and open access to data with very short revisit times, high spatial resolution, and good spectral resolution are crucial for many applications. The portfolio of possible products is vast - use-cases of such a service range from plant health monitoring, land and water body change, flood monitoring, disaster mapping and more.However the current gap between Sentinel source data and its end-users is large:• \x90  ESA’s complex Scientific Data Hub• \x90  raster files are compressed with JPEG2000 (13 raster filesfor each product, one per spectral band)• \x90  terabytes of data per week• \x90  additional processing requirementsTackling the data in an old-fashioned way -  offering individual derivative products simply does not work anymore, the associated time and costs are large and defeat most of the major benefits of the Sentinel project.Our approach combines cloud-based GIS technologies

In [9]:
list(wms.contents)  #see list of options for types of data we can get.
#wms['0'].title
#wms['0'].styles
#wms['0'].boundingBox
#wms['0'].crsOptions
#wms['0'].boundingBoxWGS84

['AGRICULTURE',
 'ARI1',
 'ARI2',
 'ATMOSPHERIC_PENETRATION',
 'B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B10',
 'B11',
 'B12',
 'B8A',
 'BAI',
 'BATHYMETRIC',
 'CHL_RED_EDGE',
 'CRI1',
 'CRI2',
 'EVI',
 'EVI2',
 'FALSE_COLOR',
 'FALSE_COLOR_URBAN',
 'GEOLOGY',
 'GRVI1',
 'LAI_SAVI',
 'MOISTURE_INDEX',
 'MSAVI2',
 'NBR_RAW',
 'NDVI',
 'NDVI_GRAY',
 'NDVI_GREEN_GRAY',
 'NDWI',
 'PSRI',
 'PSRI_NIR',
 'RED_EDGE_NDVI',
 'RE_NDWI',
 'RGB_11_8_3',
 'RGB_4_3_1',
 'RGB_8_11_12',
 'RGB_8_11_4',
 'RGB_8_5_4',
 'RGB_8_6_4',
 'SAVI',
 'SWIR',
 'TRUE_COLOR']

In [10]:
wms['TRUE_COLOR'].styles 

{'default': {'title': 'default'}}

In [11]:
[op.name for op in wms.operations]

['GetCapabilities', 'GetMap', 'GetFeatureInfo']

In [12]:
wms.getOperationByName('GetMap').methods

[{'type': 'Get',
  'url': 'http://services.sentinel-hub.com/ogc/wms/45271f49-0dcb-4329-8b35-8b6508fee50b?'}]

In [13]:
wms.getOperationByName('GetMap').formatOptions

['image/png',
 'image/jpeg',
 'image/jpg',
 'image/tiff',
 'application/vnd.google-earth.kmz+xml',
 'image/tiff;depth=8',
 'image/tiff;depth=16',
 'image/tiff;depth=32f',
 'application/vnd.google-earth.kmz+xml;image_type=image/jpeg',
 'application/vnd.google-earth.kmz+xml;image_type=image/png',
 'application/x-esri-shape',
 'application/json']

### Image Retrieval

In [14]:
import shapely

In [ ]:
#Sample format
img = wms.getmap(    layers=['NDVI_GREEN_GRAY'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=(-112.918670, 40.811920, -112.909857, 40.818094),
                     size=(300, 250),
                     format='image/png',
                     time ='2019-08-16/2019-08-21/P1D',                   
                     )

#### Image Retrieval Using Downloaded Perimeter data 

In [15]:
#Sample with first element of polygon dataframe
polygon = gp.head()['geometry'][0]
import shapely


ade = polygon.simplify(0.1, preserve_topology=True)  #simplify polygon to have fewer sides.
print(ade)
print(ade.bounds)  #coordinates of bounding box of that polygon.

POLYGON ((-112.910228325 40.81808935800009, -112.915617637 40.81276043600008, -112.918463881 40.81238991200006, -112.91863965 40.81511952100005, -112.910228325 40.81808935800009))
(-112.91863964999999, 40.812389912000064, -112.91022832499999, 40.81808935800009)


In [16]:
img2 = wms.getmap(    layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=ade.bounds,
                     size=(300, 250),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2019-08-10/2019-08-21/P1D',                   
                     )

In [17]:
## Save downloaded image as whatevername.png
out = open('sentineltest8.png', 'wb')  #
out.write(img2.read())
out.close()

In [35]:
## Potential loop code for all shapes. Untested code

def getDownloadImage(polygonbounds, timeparams, isFire, i, year):
    
    img2 = wms.getmap(layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polygonbounds,
                     size=(256, 256),  
                     format='image/png',
                     time =timeparams,                   
                     )
    filename = 'sentinel' + year + isFire + '_' + str(i) + '.png'
   
    out = open(filename, 'wb')  #
    out.write(img2.read())
    out.close()
    
def getSearchInterval(gp,i):
    from datetime import datetime
    firedate = gp['perDatTi_1'][i]
    fireyear = firedate[:9]
    todaysdate = datetime.strptime(fireyear, '%m/%d/%Y')
    todaystr = todaysdate.strftime("%Y-%m-%d")
    import datetime
    enddate = todaysdate + datetime.timedelta(days=10)  #can reduce
    enddatenow = enddate.now()
    enddatestr = enddatenow.strftime("%Y-%m-%d")
    search_interval = todaystr + '/' + enddatestr + '/' + 'P1D'
    return (search_interval, str(todaysdate.strftime("%Y")))
    
def downloadAllBurnImages(gp):
    isFire = '1'
    for i in range(len(gp)):
        polygon = gp['geometry'][i]
        polygon_simple = polygon.simplify(0.1, preserve_topology=True) 
        polygonbounds = polygon_simple.bounds
        (timeparams, year) = getSearchInterval(gp,i)
        getDownloadImage(polygonbounds, timeparams, isFire, i, year)
        
    
        
    

In [58]:
#gp.iloc[7,:]

In [55]:
downloadAllBurnImages(gp)

In [18]:
import os 
os. getcwd() 

'/Users/ola/Downloads'

In [19]:
path = '/Users/ola/Downloads/CS230Data'

In [56]:
os.chdir(path)

In [ ]:
img2 = wms.getfeatureinfo(    layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=ade.bounds,
                     size=(300, 250),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2019-08-10/2019-08-21/P1D',                   
                     )

In [ ]:
img2 = wms.getfeatureinfo( layers=['NDVI_GRAY'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=ade.bounds,
                     size=(300, 250),
                     geometry = ade,
                     format='image/png',
                     transparent = False,
                     time ='2019-08-10/2019-08-21/P1D',                   
                     )